In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install pandas

In [ ]:
import pandas as pd
from datasets import load_dataset, Image, Dataset, DatasetDict
data_path = "/content/drive/MyDrive/Seminar2/data/ribe_512x768"
dataset = load_dataset("imagefolder", data_dir=data_path)
splits = dataset["train"].train_test_split(test_size=0.33)
dataset["train"] = splits["train"]
dataset["val"] = splits["test"]
dataset

Resolving data files:   0%|          | 0/1332 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1434 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 892
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1434
    })
    val: Dataset({
        features: ['image', 'label'],
        num_rows: 440
    })
})

In [ ]:
dataset['train'][0:5]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x512 at 0x7FEF4C7E61F0>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x512 at 0x7FEF88109C40>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x512 at 0x7FEF4C7C6550>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x512 at 0x7FEF4C7C6460>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x512 at 0x7FEF4C7C6370>],
 'label': [1, 2, 0, 2, 2]}

In [ ]:
dataset['train'].features['label']

ClassLabel(names=['divlje', 'tunakavez', 'uzgoj'], id=None)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoImageProcessor

model_name_or_path = "microsoft/swin-tiny-patch4-window7-224"
feature_extractor = AutoImageProcessor.from_pretrained(model_name_or_path)
feature_extractor

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [ ]:
def process_example(example):
    inputs = feature_extractor(example['image'], return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

process_example(dataset['train'][0])

{'pixel_values': tensor([[[[ 0.0353,  0.0745,  0.1059,  ...,  0.0745,  0.0588,  0.0588],
          [ 0.0902,  0.0902,  0.0980,  ...,  0.0667,  0.0667,  0.0745],
          [ 0.1137,  0.1137,  0.0902,  ...,  0.0431,  0.0275,  0.0431],
          ...,
          [ 0.0902,  0.1059,  0.0824,  ...,  0.0588,  0.0667,  0.0667],
          [ 0.0980,  0.1216,  0.0824,  ...,  0.0588,  0.0510,  0.0510],
          [ 0.1216,  0.1294,  0.0902,  ...,  0.0510,  0.0431,  0.0353]],

         [[-0.0118,  0.0431,  0.0824,  ...,  0.0353,  0.0196,  0.0275],
          [ 0.0588,  0.0588,  0.0824,  ...,  0.0353,  0.0275,  0.0431],
          [ 0.0745,  0.0824,  0.0745,  ...,  0.0118, -0.0039,  0.0118],
          ...,
          [ 0.0667,  0.0902,  0.0667,  ...,  0.0196,  0.0353,  0.0196],
          [ 0.0667,  0.1059,  0.0667,  ...,  0.0275,  0.0196,  0.0118],
          [ 0.0902,  0.1137,  0.0745,  ...,  0.0118,  0.0118,  0.0039]],

         [[ 0.0275,  0.0588,  0.0980,  ...,  0.0039, -0.0118,  0.0039],
          [ 0

In [ ]:
def transform(example_batch):
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

dataset = dataset.with_transform(transform)

dataset['train'][0:2]

{'pixel_values': tensor([[[[ 0.0353,  0.0745,  0.1059,  ...,  0.0745,  0.0588,  0.0588],
          [ 0.0902,  0.0902,  0.0980,  ...,  0.0667,  0.0667,  0.0745],
          [ 0.1137,  0.1137,  0.0902,  ...,  0.0431,  0.0275,  0.0431],
          ...,
          [ 0.0902,  0.1059,  0.0824,  ...,  0.0588,  0.0667,  0.0667],
          [ 0.0980,  0.1216,  0.0824,  ...,  0.0588,  0.0510,  0.0510],
          [ 0.1216,  0.1294,  0.0902,  ...,  0.0510,  0.0431,  0.0353]],

         [[-0.0118,  0.0431,  0.0824,  ...,  0.0353,  0.0196,  0.0275],
          [ 0.0588,  0.0588,  0.0824,  ...,  0.0353,  0.0275,  0.0431],
          [ 0.0745,  0.0824,  0.0745,  ...,  0.0118, -0.0039,  0.0118],
          ...,
          [ 0.0667,  0.0902,  0.0667,  ...,  0.0196,  0.0353,  0.0196],
          [ 0.0667,  0.1059,  0.0667,  ...,  0.0275,  0.0196,  0.0118],
          [ 0.0902,  0.1137,  0.0745,  ...,  0.0118,  0.0118,  0.0039]],

         [[ 0.0275,  0.0588,  0.0980,  ...,  0.0039, -0.0118,  0.0039],
          [ 0

In [ ]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [ ]:
from transformers.trainer_utils import EvalPrediction
import numpy as np
from datasets import load_metric
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred: EvalPrediction):
  preds = np.argmax(eval_pred.predictions, axis=1)
  return {
    "acc": accuracy_score(eval_pred.label_ids, preds),
    "f1": f1_score(eval_pred.label_ids, preds, average="weighted"),
    "precision": precision_score(eval_pred.label_ids, preds, average="weighted"),
    "recall": recall_score(eval_pred.label_ids, preds, average="weighted")
    }


In [ ]:
from transformers import AutoModelForImageClassification

labels = dataset['train'].features['label'].names

model = AutoModelForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/Seminar2/model",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=7,
  fp16=True,
  save_steps=60,
  eval_steps=60,
  logging_steps=60,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=feature_extractor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
60,0.177400,0.111885,0.970455,0.970612,0.970876,0.970455
120,0.069200,0.090298,0.965909,0.965723,0.967357,0.965909
180,0.012900,0.058611,0.984091,0.984007,0.984066,0.984091
240,0.021700,0.052177,0.984091,0.983996,0.984383,0.984091
300,0.003900,0.052779,0.988636,0.988672,0.988876,0.988636
360,0.003200,0.048039,0.990909,0.990928,0.991023,0.990909


***** train metrics *****
  epoch                    =         7.0
  total_flos               = 450633764GF
  train_loss               =      0.0444
  train_runtime            =  0:05:15.04
  train_samples_per_second =       19.82
  train_steps_per_second   =       1.244


In [ ]:
metrics = trainer.evaluate(dataset['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        7.0
  eval_acc                =     0.5662
  eval_f1                 =     0.5327
  eval_loss               =     2.0566
  eval_precision          =     0.6056
  eval_recall             =     0.5662
  eval_runtime            = 0:03:24.11
  eval_samples_per_second =      7.025
  eval_steps_per_second   =      0.882
